In [1]:
from flask import Flask, render_template, redirect, request, url_for, session
from flask_mysqldb import MySQL, MySQLdb
import bcrypt
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import io
import random
from flask import Response
from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas
from matplotlib.figure import Figure
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

In [2]:
data = pd.read_csv('ENCODE.csv')

In [3]:
data.dtypes

Jenis_Alat            int64
MEREK                 int64
BUATAN                int64
TAHUN_PEMBUATAN       int64
KATEGORI_KERUSAKAN    int64
KONDISI               int64
Sertifikasi           int64
Keterangan            int64
dtype: object

In [5]:
data.head(10)

,Jenis_Alat,MEREK,BUATAN,TAHUN_PEMBUATAN,KATEGORI_KERUSAKAN,KONDISI,Sertifikasi,Keterangan
0,1,1,1,2,3,53,1,0
1,1,2,1,6,4,80,1,2
2,1,2,1,6,2,80,1,1
3,1,1,1,7,4,87,1,2
4,1,1,1,7,4,87,1,2
5,1,1,1,7,2,87,1,2
6,1,1,1,9,1,100,0,2
7,1,2,1,6,2,80,1,1
8,1,2,1,6,1,80,1,1
9,1,2,1,1,3,47,1,0


In [6]:
data.columns

Index(['Jenis_Alat', 'MEREK', 'BUATAN', 'TAHUN_PEMBUATAN',
       'KATEGORI_KERUSAKAN', 'KONDISI ', 'Sertifikasi ', 'Keterangan'],
      dtype='object')

In [8]:
data.describe()

,Jenis_Alat,MEREK,BUATAN,TAHUN_PEMBUATAN,KATEGORI_KERUSAKAN,KONDISI,Sertifikasi,Keterangan
count,546.000000,546.000000,546.000000,546.000000,546.000000,546.000000,546.000000,546.000000
mean,10.358974,12.745421,7.223443,6.351648,2.954212,74.553114,0.996337,1.705128
std,2.940898,3.409004,2.006750,1.235918,1.446194,26.265579,0.060467,0.617075
min,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000
25%,9.000000,12.000000,5.000000,6.000000,2.000000,40.000000,1.000000,2.000000
50%,11.000000,14.000000,8.000000,6.000000,3.000000,80.000000,1.000000,2.000000
75%,13.000000,16.000000,9.000000,6.000000,4.000000,100.000000,1.000000,2.000000
max,14.000000,16.000000,10.000000,10.000000,24.000000,100.000000,1.000000,2.000000


In [10]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 546 entries, 0 to 545
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype
---  ------              --------------  -----
 0   Jenis_Alat          546 non-null    int64
 1   MEREK               546 non-null    int64
 2   BUATAN              546 non-null    int64
 3   TAHUN_PEMBUATAN     546 non-null    int64
 4   KATEGORI_KERUSAKAN  546 non-null    int64
 5   KONDISI             546 non-null    int64
 6   Sertifikasi         546 non-null    int64
 7   Keterangan          546 non-null    int64
dtypes: int64(8)
memory usage: 34.2 KB


In [11]:
data_features = data.drop('Keterangan', axis=1)
data_target = data['Keterangan']

In [12]:
data_features

,Jenis_Alat,MEREK,BUATAN,TAHUN_PEMBUATAN,KATEGORI_KERUSAKAN,KONDISI,Sertifikasi
0,1,1,1,2,3,53,1
1,1,2,1,6,4,80,1
2,1,2,1,6,2,80,1
3,1,1,1,7,4,87,1
4,1,1,1,7,4,87,1
...,...,...,...,...,...,...,...
541,14,16,5,6,4,100,1
542,14,16,5,6,4,95,1
543,14,16,5,6,4,100,1
544,14,16,5,6,4,100,1


In [13]:

from sklearn.preprocessing import StandardScaler
scale = StandardScaler().fit_transform(data_features)

cols = list(data_features.columns)

data_features_scale = pd.DataFrame(scale, columns=cols)
data_features_scale.head()

,Jenis_Alat,MEREK,BUATAN,TAHUN_PEMBUATAN,KATEGORI_KERUSAKAN,KONDISI,Sertifikasi
0,-3.185271,-3.448571,-3.104099,-3.524214,0.031690,-0.821336,0.060634
1,-3.185271,-3.154961,-3.104099,-0.284785,0.723794,0.207568,0.060634
2,-3.185271,-3.154961,-3.104099,-0.284785,-0.660414,0.207568,0.060634
3,-3.185271,-3.448571,-3.104099,0.525072,0.723794,0.474320,0.060634
4,-3.185271,-3.448571,-3.104099,0.525072,0.723794,0.474320,0.060634


In [15]:
X_train, X_test, y_train, y_test = train_test_split(data_features_scale, data_target, test_size=0.2, random_state=42)

print('Train set :', X_train.shape, y_train.shape)
print('Test set :', X_test.shape, y_test.shape)

Train set : (436, 7) (436,)
Test set : (110, 7) (110,)


In [16]:
#Membuat model naive bayes terhadap training test
from sklearn.naive_bayes import GaussianNB
classifier = GaussianNB()
classifier.fit(X_train, y_train)

GaussianNB()

In [17]:
#prediksi hasil test set
y_pred = classifier.predict(X_test)
y_pred

array([2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 1, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2,
       2, 2, 2, 1, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2,
       2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 0, 2, 1, 2,
       2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 1, 2, 1, 2, 1, 2, 2, 2, 2, 1, 2, 2,
       1, 1, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 1, 2, 1, 1, 2, 1],
      dtype=int64)

In [18]:

print(y_pred[0:5])
print(y_test[0:5])

[2 2 2 2 2]
316    2
77     0
360    2
90     2
495    2
Name: Keterangan, dtype: int64


In [19]:
#menghitung akurasi
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)

0.7090909090909091

In [27]:
import seaborn as sns

plt.figure(figsize=(10, 6))
sns.countplot(data=data, x="2")
plt.title("Kepenuhan Penumpang", fontsize=20)
plt.show()

ValueError: Could not interpret input '2'

<Figure size 720x432 with 0 Axes>